# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [2]:
def nhl_correlation(): 
    
    nhl_df = pd.read_csv("assets/nhl.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    def get_city_name(team_name: str) -> str:
        """
        A function that splits the team names to extract the City names.
        In most cases the city name was the first word of the team name followed by another word that identifies the team,
        but some teams had names with more than two words where the team name consisted two words.
        """
        
        # adjectives in the names to skip during parsing
        _skips = ["Red", "Blue", "Golden", "Maple"]
        _names = team_name.split(" ")
        if len(_names) > 2:
            if _names[1] not in _skips:
                return _names[0] + " " + _names[1]
            else:
                return _names[0]
        elif len(_names) == 2:
            return _names[0]
        else:
            return None
    
    # select just the 2018 data & the "team", "W", "L" columns.
    # there are some row lables [0, 9, 18, 26], drop them since we do not need them
    nhl_df = nhl_df.query("year == 2018").loc[:, ["team", "W", "L"]].drop([0, 9, 18, 26], axis = 0).reset_index(drop = True)
    
    # create a new column using the parsed results
    nhl_df["Metropolitan area"] = nhl_df.team.apply(get_city_name)
    
    # compute the win/loss ratio & append that to the dataframe as a column
    nhl_df["wl_ratio"] = nhl_df.W.astype(np.int64) / (nhl_df.W.astype(np.int64) + nhl_df.L.astype(np.int64))
    
    # make a new dataframe with just the columns that we are interested in & type cast the population column as integers
    _cities = cities.loc[:, ["Metropolitan area", "Population (2016 est.)[8]"]]
    _cities["Population (2016 est.)[8]"]  = _cities.loc[:, "Population (2016 est.)[8]"].astype(np.int64)
    
    # a lookup dictionary to replace city names in the nhl dataframe according to the city names in the cities datafarme.
    rename_dict = {
    "New York": "New York City",
    "Tampa Bay": "Tampa Bay Area",
    "Washington": "Washington, D.C.",
    "Dallas": "Dallas–Fort Worth",
    "Vegas": "Las Vegas",
    "Colorado": "Denver",
    "San Jose": "San Francisco Bay Area",
    "Anaheim": "Los Angeles",
    "Florida": "Miami–Fort Lauderdale",
    "Arizona": "Phoenix",
    "Carolina": "Raleigh",
    "Minnesota": "Minneapolis–Saint Paul",
    "New Jersey": "New York City"
    }
    
    # renaming the city names in nhl dataframe according to the cities dataframe, using our lookup dictionary.
    nhl_df["Metropolitan area"] = nhl_df.loc[:, "Metropolitan area"] \
                .apply(lambda name: rename_dict.get(name) if name in rename_dict.keys() else name)
    
    # merge the two dataframes using Metropolitan area as the key
    nhl_merge = pd.merge(_cities, nhl_df, how = "inner", on = "Metropolitan area")
    # this contains three records for NYC, two for LA with others being unique.
    
    # average the numeric columns for repetitive city records.
    final = nhl_merge.groupby("Metropolitan area").agg({"Population (2016 est.)[8]": np.mean, "wl_ratio": np.mean})
    
    population_by_region = final.loc[:, "Population (2016 est.)[8]"].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = final.loc[:, "wl_ratio"].to_list()   # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [3]:
nhl_correlation()

0.012486162921209907

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [6]:
def nba_correlation():
    
    nba_df = pd.read_csv("assets/nba.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]
    
    def get_city_name(team_name: str) -> str:
        """
        A function that splits the team names to extract the City names.
        In most cases the city name was the first word of the team name followed by another word that identifies the team,
        but some teams had names with more than two words where the team name consisted two words.
        """
        
        # adjectives in the names to skip during parsing
        _skips = ["Trail", "City"]
        _names = team_name.split(" ")
        if len(_names) > 2:
            if _names[1] not in _skips:
                return _names[0] + " " + _names[1]
            else:
                return _names[0]
        elif len(_names) == 2:
            return _names[0]
        else:
            return None
    
    # create a new column using the parsed results
    nba_df["Metropolitan area"] = nba_df.team.apply(get_city_name)
    
    # select just the 2018 data & the "team", "W", "L" columns.
    nba_df = nba_df.query("year == 2018").loc[:, ["team", "W", "L", "Metropolitan area"]].astype({"W": np.int64, "L": np.int64})
    

    
    # compute the win/loss ratio & append that to the dataframe as a column
    nba_df["wl_ratio"] = nba_df.W / (nba_df.W + nba_df.L)
    
    # make a new dataframe with just the columns that we are interested in & type cast the population column as integers
    _cities = cities.loc[:, ["Metropolitan area", "Population (2016 est.)[8]"]]
    _cities["Population (2016 est.)[8]"]  = _cities.loc[:, "Population (2016 est.)[8]"].astype(np.int64)
    
    # a lookup dictionary to replace city names in the nba dataframe according to the city names in the cities datafarme.
    rename_dict = {
    "New York": "New York City",
    "Brooklyn": "New York City",
    "Tampa Bay": "Tampa Bay Area",
    "Washington": "Washington, D.C.",
    "Dallas": "Dallas–Fort Worth",
    "Vegas": "Las Vegas",
    "Colorado": "Denver",
    "San Jose": "San Francisco Bay Area",
    "Anaheim": "Los Angeles",
    "Florida": "Miami–Fort Lauderdale",
    "Arizona": "Phoenix",
    "Carolina": "Raleigh",
    "Minnesota": "Minneapolis–Saint Paul",
    "New Jersey": "New York City",
    "Indiana": "Indianapolis",
    "Miami": "Miami–Fort Lauderdale",
    "Oklahoma": "Oklahoma City",
    "Utah": "Salt Lake City",
    "Golden State": "San Francisco Bay Area"
    }
    
    # renaming the city names in nba dataframe according to the cities dataframe, using our lookup dictionary.
    nba_df["Metropolitan area"] = nba_df.loc[:, "Metropolitan area"] \
                .apply(lambda name: rename_dict.get(name) if name in rename_dict.keys() else name)
    
    # merge the two dataframes using Metropolitan area as the key
    nba_merge = pd.merge(_cities, nba_df, how = "inner", on = "Metropolitan area")
    
    
    # average the numeric columns for repetitive city records.
    final = nba_merge.groupby("Metropolitan area").agg({"Population (2016 est.)[8]": np.mean, "wl_ratio": np.mean})
    
    population_by_region = final.loc[:, "Population (2016 est.)[8]"].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = final.loc[:, "wl_ratio"].to_list()   # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [7]:
nba_correlation()

-0.17657160252844623

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [8]:
def mlb_correlation(): 
     
    mlb_df = pd.read_csv("assets/mlb.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]

    
    def get_city_name(team_name: str) -> str:
        """
        A function that splits the team names to extract the City names.
        In most cases the city name was the first word of the team name followed by another word that identifies the team,
        but some teams had names with more than two words where the team name consisted two words.
        """
        
        # adjectives in the names to skip during parsing
        _skips = ["Red", "Blue", "White"]
        _names = team_name.split(" ")
        if len(_names) > 2:
            if _names[1] not in _skips:
                return _names[0] + " " + _names[1]
            else:
                return _names[0]
        elif len(_names) == 2:
            return _names[0]
        else:
            return None
    
    # create a new column using the parsed results
    mlb_df["Metropolitan area"] = mlb_df.team.apply(get_city_name)
    
    # select just the 2018 data & the "team", "W", "L" columns.
    mlb_df = mlb_df.query("year == 2018").astype({"W": np.int64, "L": np.int64}).loc[:, ["Metropolitan area", "team",  "W", "L"]]
    
    # compute the win/loss ratio & append that to the dataframe as a column
    mlb_df["wl_ratio"] = mlb_df.W / (mlb_df.W + mlb_df.L)
    
    # make a new dataframe with just the columns that we are interested in & type cast the population column as integers
    _cities = cities.loc[:, ["Metropolitan area", "Population (2016 est.)[8]"]]
    _cities["Population (2016 est.)[8]"]  = _cities.loc[:, "Population (2016 est.)[8]"].astype(np.int64)
    
    # a lookup dictionary to replace city names in the mlb dataframe according to the city names in the cities datafarme.
    rename_dict = {
    "New York": "New York City",
    "Brooklyn": "New York City",
    "Tampa Bay": "Tampa Bay Area",
    "Washington": "Washington, D.C.",
    "Dallas": "Dallas–Fort Worth",
    "Vegas": "Las Vegas",
    "Colorado": "Denver",
    "San Jose": "San Francisco Bay Area",
    "Anaheim": "Los Angeles",
    "Florida": "Miami–Fort Lauderdale",
    "Arizona": "Phoenix",
    "Carolina": "Raleigh",
    "Minnesota": "Minneapolis–Saint Paul",
    "New Jersey": "New York City",
    "Indiana": "Indianapolis",
    "Miami": "Miami–Fort Lauderdale",
    "Oklahoma": "Oklahoma City",
    "Utah": "Salt Lake City",
    "Golden State": "San Francisco Bay Area",
    "San Francisco": "San Francisco Bay Area",
    "Oakland": "San Francisco Bay Area",
    "Texas": "Dallas–Fort Worth"
    }
    
    # renaming the city names in nba dataframe according to the cities dataframe, using our lookup dictionary.
    mlb_df["Metropolitan area"] = mlb_df.loc[:, "Metropolitan area"] \
                .apply(lambda name: rename_dict.get(name) if name in rename_dict.keys() else name)
    
    # merge the two dataframes using Metropolitan area as the key
    mlb_merge = pd.merge(_cities, mlb_df, how = "inner", on = "Metropolitan area")
    
    
    # average the numeric columns for repetitive city records.
    final = mlb_merge.groupby("Metropolitan area").agg({"Population (2016 est.)[8]": np.mean, "wl_ratio": np.mean})

    population_by_region = final.loc[:, "Population (2016 est.)[8]"].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = final.loc[:, "wl_ratio"].to_list()   # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [9]:
mlb_correlation()

0.15027698302669307

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [10]:
def nfl_correlation(): 
    
    
    nfl_df = pd.read_csv("assets/nfl.csv")
    cities = pd.read_html("assets/wikipedia_data.html")[1]
    cities = cities.iloc[:-1,[0,3,5,6,7,8]]

    
    def get_city_name(team_name: str) -> str:
        """
        A function that splits the team names to extract the City names.
        In most cases the city name was the first word of the team name followed by another word that identifies the team,
        but some teams had names with more than two words where the team name consisted two words.
        """
        _names = team_name.split(" ")
        if len(_names) > 2:
            return _names[0] + " " + _names[1]
        elif len(_names) == 2:
            return _names[0]
        else:
            return None
    
    # create a new column using the parsed results
    nfl_df["Metropolitan area"] = nfl_df.team.apply(get_city_name)
    
    # select just the 2018 data & the "team", "W", "L" columns.
    nfl_df = nfl_df.loc[:, ["team", "W", "L", "year", "Metropolitan area"]].query("year == 2018").drop(np.arange(0, 36, step = 5), axis = 0)
    nfl_df = nfl_df.astype({"W": np.int64, "L": np.int64})
    
    # compute the win/loss ratio & append that to the dataframe as a column
    nfl_df["wl_ratio"] = nfl_df.W / (nfl_df.W + nfl_df.L)
    
    # make a new dataframe with just the columns that we are interested in & type cast the population column as integers
    _cities = cities.loc[:, ["Metropolitan area", "Population (2016 est.)[8]"]]
    _cities["Population (2016 est.)[8]"]  = _cities.loc[:, "Population (2016 est.)[8]"].astype(np.int64)
    
    # a lookup dictionary to replace city names in the mlb dataframe according to the city names in the cities datafarme.
    rename_dict = {
    "New York": "New York City",
    "Brooklyn": "New York City",
    "Tampa Bay": "Tampa Bay Area",
    "Washington": "Washington, D.C.",
    "Dallas": "Dallas–Fort Worth",
    "Vegas": "Las Vegas",
    "Colorado": "Denver",
    "San Jose": "San Francisco Bay Area",
    "Anaheim": "Los Angeles",
    "Florida": "Miami–Fort Lauderdale",
    "Arizona": "Phoenix",
    "Carolina": "Raleigh",
    "Minnesota": "Minneapolis–Saint Paul",
    "New Jersey": "New York City",
    "Indiana": "Indianapolis",
    "Miami": "Miami–Fort Lauderdale",
    "Oklahoma": "Oklahoma City",
    "Utah": "Salt Lake City",
    "Golden State": "San Francisco Bay Area",
    "San Francisco": "San Francisco Bay Area",
    "Oakland": "San Francisco Bay Area",
    "Texas": "Dallas–Fort Worth",
    "New England": "Boston",
    "Tennessee": "Nashville"
    }
    
    # renaming the city names in nba dataframe according to the cities dataframe, using our lookup dictionary.
    nfl_df["Metropolitan area"] = nfl_df.loc[:, "Metropolitan area"] \
                .apply(lambda name: rename_dict.get(name) if name in rename_dict.keys() else name)
    
    # merge the two dataframes using Metropolitan area as the key
    nfl_merge = pd.merge(_cities, nfl_df, how = "inner", on = "Metropolitan area")
    
    
    # average the numeric columns for repetitive city records.
    final = nfl_merge.groupby("Metropolitan area").agg({"Population (2016 est.)[8]": np.mean, "wl_ratio": np.mean})

    population_by_region = final.loc[:, "Population (2016 est.)[8]"].to_list() # pass in metropolitan area population from cities
    win_loss_by_region = final.loc[:, "wl_ratio"].to_list()   # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

In [11]:
nfl_correlation()

0.008975826138476653

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df = pd.read_csv("assets/mlb.csv")
nba_df = pd.read_csv("assets/nhl.csv")
nba_df = pd.read_csv("assets/nba.csv")
nfl_df = pd.read_csv("assets/nfl.csv")
cities = pd.read_html("assets/wikipedia_data.html")[1]
cities = cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values